 # **AKER SV: Demo #1 (Project from Scratch)**
 
 This notebook will outline the basic functionality of **AKER SV** by building a verification project from scratch.

# Installing/Importing AKER SV

In [1]:
#Install aker_sv from local files
%cd aker_sv
%pip install .
%cd ../

In [2]:
import aker_sv as aker

# Creating an Empty Project

The following cell creates an empty **AKER SV** project. Refer to the final section of this python notebook or (Demo #2) to see another way of creating a project.

In [3]:
my_proj = aker.AKERProj()

# Defining Threat Model (Optional)

**AKER SV** enables you to define a threat model for your project. This threat model has no impact on the automated property generation process (i.e., template expansion). However, the threat model will be present in the auto-generated html documentation for the project (available in version 0.1.1). 

The following cell demonstrates defining a threat model for your project.

In [4]:
example_threat_model = \
"We consider an integrity scenario where C is untrusted and the ACW and P are trusted.\
Therefore, the threat model considers C's ability to communicate with P via the ACW in\
a manner which does not adhere to the statically-configured LACP"

my_proj.set_threat_model(example_threat_model)

# Creating a Security Property

Among other things, an **AKER SV** project stores a collection of security property objects `aker_sv.SecProp()`. Each `SecProp()` object has the following fields:
* `id` - a string specifying an ID
* `CWEs` - a list of strings where each element is an ID for a relevant Mitre CWE
* `requirement` - a string specifying a plain-language security requirement
* `template` - a string specifying a security property template with generic signals/values to replace
* `type` - a string specifying the type of the security property (auto-generated based on provided template [IFT or Trace])
* `assets` - a list where each element is a tuple of n strings (n is equal to the number of generic signals/values to replace)
* `expansion_total` - an integer specifying the total number of expanded security properties (auto-generated after template expansion)

Of these fields, the only fields that must be set in order to generate specific properties from a property template are: `id`, `template`, and `assets`.

The following cells demonstrate:
* creating an empty `SecProp()` object
* setting the object's required fields
* generating specific properties via template expansion

The first cell defines an information flow tracking (IFT) security property and the second cell defines a regular trace/functional security property. Properties are specified in Tortuga Logic's Sentinel security language. 

In [5]:
#####################
# Multi Line Approach + write generated SPs to console
#####################
sp_1 = aker.SecProp()

sp_1.set_id("01")

template =\
'''
SP01_GENERIC: assert iflow(
  `sig_x` =/=> `sig_y`
);
'''
sp_1.set_template(template)

assets = [
  ("sig_a", "sig_b"),
  ("sig_c", "sig_d"),
  ("sig_e", "sig_f"), 
  ("sig_g", "sig_h"),
  ("sig_i", "sig_j"),
]
sp_1.set_assets(assets)

sp_1.expand_template()

######################
# Single Line Approach + write generated SPs to file
######################
aker.SecProp(sp_id="01", temp=template, assets=assets).expand_template("SP_01.as")

SP01_0: assert iflow(
  sig_a =/=> sig_b
);


SP01_1: assert iflow(
  sig_c =/=> sig_d
);


SP01_2: assert iflow(
  sig_e =/=> sig_f
);


SP01_3: assert iflow(
  sig_g =/=> sig_h
);


SP01_4: assert iflow(
  sig_i =/=> sig_j
);




In [6]:
#####################
# Multi Line Approach + write generated SPs to console
#####################
sp_2 = aker.SecProp()

sp_2.set_id("02")

template =\
'''
SP02_GENERIC: assert iflow(
  `sig_x` == `sig_y`
);
'''
sp_2.set_template(template)

assets = [
  ("sig_a", "sig_b"),
  ("sig_c", "sig_d"),
  ("sig_e", "sig_f"), 
  ("sig_g", "sig_h"),
  ("sig_i", "sig_j"),
]
sp_2.set_assets(assets)

sp_2.expand_template()

######################
# Single Line Approach + write generated SPs to file
######################
aker.SecProp(sp_id="02", temp=template, assets=assets).expand_template("SP_02.as")

SP02_0: assert iflow(
  sig_a == sig_b
);


SP02_1: assert iflow(
  sig_c == sig_d
);


SP02_2: assert iflow(
  sig_e == sig_f
);


SP02_3: assert iflow(
  sig_g == sig_h
);


SP02_4: assert iflow(
  sig_i == sig_j
);




# Adding a Security Property to an AKER Project

As mentioned, an **AKER SV** project stores a collection of security property objects `aker_sv.SecProp()`. More specifically, these SPs are stored in a python dictionary `AKERProj().SPs` where an SP's `id` is the key and the SP (i.e., the `SecProp()` object itself) is the value.

Adding an SP to your project is as simple as calling your project's `AKERProj().add_sec_prop()` function. Although you can directly insert key-value pairs into the SP dictionary, like so `AKERProj().SPs[key] = value`, it is highly recommend that you utilize `add_sec_prop()` because of the built-in protections/checks that are performed.  


The following cells demonstrate:
* adding multiple security properties to a project 

In [7]:
my_proj.add_sec_prop(sp_1)
my_proj.add_sec_prop(sp_2)

# Performing Template Expansion with an AKER Project

If you would like to perform template expansion on all SPs in a AKER Project at once, you can call your project's `AKERProj().expand_templates()` function.

Note that if any SP does not have its `id`, `template`, and `assets` field set the property will be skipped during template expansion.

By default, `expand_templates()` will create an output directory with the prefix `aker_sps` in the current working directory. This directory will contain a `.as` file for each SP object that is expanded.  

In [8]:
my_proj.expand_templates()

[AKERProj.expand_templates] No output_path specified. Creating output at ./aker_sps


# Exporting/Importing an AKER Project

**AKER SV** projects support exporting to and importing from a csv file. 

To export a project to csv, call the project's `AKERProj().to_csv()` function with an output file path string as input. The resulting `.csv` will be structured as shown in the table below. Certain cells may be empty if certain SP fields were not set or if template expansion was not performed. 

SP ID | Related CWEs | SP Requirement | SP Specification Template | SP Type | SP Assets | Total SPs After Expanding|
------|--------------|----------------|---------------------------|---------|-----------|--------------------------|
------|--------------|----------------|---------------------------|---------|-----------|--------------------------|
------|--------------|----------------|---------------------------|---------|-----------|--------------------------|


To import a project from csv, call the `AKERProj().from_csv()` function with the csv file path string as input. The imported `.csv` must be structured as shown in one of the two table below. Cells can be empty but the first 4 or 5 columns of the header row must match what is shown.   

SP ID | Related CWEs | SP Requirement | SP Specification Template | SP Type |
------|--------------|----------------|---------------------------|---------|
------|--------------|----------------|---------------------------|---------|
------|--------------|----------------|---------------------------|---------|


SP ID | Related CWEs | SP Requirement | SP Specification Template | SP Type | SP Assets |
------|--------------|----------------|---------------------------|---------|-----------|
------|--------------|----------------|---------------------------|---------|-----------|
------|--------------|----------------|---------------------------|---------|-----------|

The following cell demonstrates:
* exporting a project
* importing a project

In [9]:
my_proj.to_csv("aker_sv_demo_1.csv", with_assets=True)

my_proj2 = aker.AKERProj("aker_sv_demo_1.csv")

# Uncomment the lines below to compare the SPs of each project to check for project equality
#my_proj2.expand_templates()
#print("Import Successful = {}".format(list(my_proj.SPs.values()) == list(my_proj2.SPs.values())))